In [2]:
using RigidBodyTreeInspector
using RigidBodyDynamics
using StaticArrays
using Plots
gr()
using CoordinateTransformations
using DataFrames, DataArrays
import SimpleGradientDescent: NaiveSolver
import Flash
import DrakeVisualizer

In [3]:
DrakeVisualizer.new_window()

Process(`/Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/../deps/usr/bin/drake-visualizer`, ProcessRunning)

In [32]:
reload("Flash")

In [35]:
urdf = "$(ENV["HOME"])/locomotion/explorations/point-cloud-signed-distance/examples/data/IRB140/urdf/irb_140_convhull.urdf"
package_path = ["$(ENV["HOME"])/locomotion/drake-distro/drake/examples"]
mechanism = parse_urdf(Float64, urdf);
vis = DrakeVisualizer.Visualizer()
DrakeVisualizer.load!(vis[:urdf], parse_urdf(urdf, mechanism; package_path=package_path))
inspect(mechanism, vis[:urdf])

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint2.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint3.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint4.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint5.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint6.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

true

In [36]:
model = Flash.Models.load_urdf(urdf, package_path=package_path)

Manipulator with 8 links and 7 surfaces

In [37]:
# Construct the sensor and draw its view rays for debugging
sensor = Flash.DepthSensors.Kinect(41, 41);
camera_origin = SVector(0., 1.5, 0.5)
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi/2, 1, 0, 0)))
Flash.DepthSensors.draw_rays(vis[:sensor], sensor, camera_tform)

true

In [38]:
state = Flash.ManipulatorState(model);
skin = Flash.skin(state);
load!(vis[:skin], DrakeVisualizer.contour_mesh(skin, [-.5, -.5, -.25], [1., .5, 1], 0.0, 0.05))

Visualizer with path prefix Symbol[:skin] using LCM LCMCore.LCM(Ptr{Void} @0x00007fab63098b70,RawFD(114),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007fab630e0860)])

In [39]:
# Difference between two angles, compensating for wraparound
# (taken from angleDiff.m in Drake)
function angle_diff(phi1, phi2)
    mod(phi2 - phi1 + pi, 2*pi) - pi;
end

angle_diff (generic function with 1 method)

In [44]:
DrakeVisualizer.delete!(vis[:skin])
DrakeVisualizer.delete!(vis[:urdf])

true

In [65]:
vis_data = parse_urdf(urdf, model.mechanism; package_path=package_path)
vis_true = vis[:actual]
vis_estimated = vis[:estimated]
DrakeVisualizer.load!(vis[:actual], vis_data)
DrakeVisualizer.load!(vis[:estimated], vis_data)

true

In [67]:
# Now let's try to use the raycast data to recover the joint angles.
# We'll do that by choosing a true (hidden) robot state:
true_state = Flash.ManipulatorState(model)
DrakeVisualizer.draw!(vis_true, true_state.mechanism_state)

# Now we can raycast that true state to get a set of sensed points:
sensed_points = Flash.DepthSensors.raycast(true_state, sensor, camera_tform)
load!(vis[:raycast], DrakeVisualizer.PointCloud(sensed_points))

Visualizer with path prefix Symbol[:raycast] using LCM LCMCore.LCM(Ptr{Void} @0x00007fab63098b70,RawFD(114),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007fab630e0860)])

In [70]:
function test_gradient_descent(model, sensed_points, true_state, loops)
    x_true = copy(Flash.flatten(true_state))
    trials = Vector{DataFrame}()
    workspace_state = Flash.ManipulatorState(model)
    
    for i = 1:loops
        errors = Float64[]
        xs = Vector{Float64}[]
        costs = Float64[]
        
        function callback{T}(x::AbstractVector{T}, c)
            # When computing errors, we'll just ignore the last three joints, since
            # they are almost impossible to track at this scale
            x_value = Flash.value.(x)
            push!(errors, norm(angle_diff.(x_value[1:end-3], x_true[1:end-3])))
            push!(xs, x_value)
            push!(costs, Flash.value(c))
            Flash.GradientDescent.unflatten!(workspace_state, x_value)
            DrakeVisualizer.draw!(vis_estimated, workspace_state.mechanism_state)
        end
        
        x_estimated = copy(x_true)
        x_estimated += pi * (rand(length(x_estimated)) - 0.5)
        Flash.Tracking.estimate_state(model, 
            sensed_points, x_estimated; callback=callback,
        solver=NaiveSolver(length(x_estimated),
                              rate=20.0,
                              max_step=0.5,
                              iteration_limit=100,
                            gradient_convergence_tolerance=1e-4))
        push!(trials, DataFrame(x=xs, err=errors, cost=costs))
    end
    
    trials
end


test_gradient_descent (generic function with 1 method)

In [72]:
trials = test_gradient_descent(model, sensed_points, true_state, 20);